## Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Carga de los datos

Link de Kaggle:

https://www.kaggle.com/justinas/nba-players-data

Dataset desde la temporada 96/97 hasta la 2019/2020 con todos los jugadores de la NBA

In [ ]:
import os;
os.listdir('/kaggle/input/')

In [ ]:
df = pd.read_csv('../input/nba-players-data/all_seasons.csv')
df = df.drop("Unnamed: 0", axis=1)

In [ ]:
print("El dataset cuenta con", df.shape[1], "columnas")

Se crea una nueva columna que describe si el jugador es extranjero o estadounidense:

In [ ]:
df["NAC_USA"] = df["country"]

In [ ]:
localizacion = df[df["NAC_USA"]!="USA"].index
df.loc[localizacion,"NAC_USA"] = "Extranjeros"

Se filtra el dataset por las últimas 4 temporadas para algunos de los gráficos

In [ ]:
df_2017 = df[df["season"]=="2016-17"]
df_2018 = df[df["season"]=="2017-18"]
df_2019 = df[df["season"]=="2018-19"]
df_2020 = df[df["season"]=="2019-20"]

## Histograma con las medias de puntos de cada jugador en cada temporada

In [ ]:
seasons = ["2016-17","2017-18", "2018-19","2019-20"]

In [ ]:
fig1 = make_subplots(rows = 2, cols = 2,
                    subplot_titles = seasons) 



fig1.add_trace(
     go.Histogram(x=df_2017.pts, name='2016/17', marker_color='#17becf'),
        row = 1, col = 1)

fig1.add_trace(
     go.Histogram(x=df_2018.pts, name='2017/18', marker_color='#1f77b4'),
        row = 1, col = 2)

fig1.add_trace(
     go.Histogram(x=df_2019.pts, name='2018/19', marker_color='#636EFA'),
        row = 2, col = 1)

fig1.add_trace(
     go.Histogram(x=df_2020.pts, name='2019/20', marker_color='#00B5F7'),
        row = 2, col = 2)

fig1.update_layout(
    title="Histograma de las medias de puntos de cada jugador por temporada"
)

fig1.show()

**Conclusiones:**
La mayor parte de los jugadores se sitúa en valores bajos de anotación, donde las medias más concurridas suelen rondar entre los 5 y 8 puntos por partido. En la temporada 2018-19 destaca la escasez de jugadores con medias por encima de los 20 puntos, dado que en otras temporadas sí ocurre esto. En general las distribuciones son bastante similares.

## Line plot con la evolución de medias de varias variables

Esta vez sí que vamos a ver la evolución en todo el dataframe

In [ ]:
df_evol = df.groupby("season")["pts", "reb","ast"].mean()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                    x = df_evol.index,
                    y = df_evol.pts,
                    mode = "lines",
                    name = "puntos",
                    text= "Puntos"))
fig.add_trace(go.Scatter(
                    x = df_evol.index,
                    y = df_evol.reb,
                    mode = "lines",
                    name = "rebotes",
                    text= "Rebotes"))
fig.add_trace(go.Scatter(
                    x = df_evol.index,
                    y = df_evol.ast,
                    mode = "lines",
                    name = "asistencias",
                    text= "Asistencias"))

fig.add_annotation(x="2009", y=7.8,
            text="Aquí se experimenta una pequeña subida",
                  showarrow = False)

fig.update_layout(
    title="Evolución de la media puntos, rebotes y asistencias")

fig.show()

**Conclusiones:**
Se puede observar que las medias de rebotes y asistencias se mantienen bastante estables a lo largo del tiempo. Sin embargo, se puede ver una ligera tendencia alcista en la media de puntos desde el 2004 hasta el 2009, aunque desde este año la media se ha ido reduciendo.

## Comparación entre las variables de puntos, asistencias y rebotes

In [ ]:
df_filt = df[df["season"].isin(seasons)]

#### Boxplots

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    y=df_filt.pts,
    x=df_filt.season,
    name='Puntos',
    marker_color='#3D9970'
))
fig.add_trace(go.Box(
    y=df_filt.ast,
    x=df_filt.season,
    name='Asistencias',
    marker_color='#FF4136'
))
fig.add_trace(go.Box(
    y=df_filt.reb,
    x=df_filt.season,
    name='Rebotes',
    marker_color='#FF851B'
))

fig.update_layout(
    title = "Boxplots comparativos entre las variables para las 4 últimas temporadas",
    yaxis_title='Medias',
    boxmode='group' 
)
fig.show()

**Conclusiones:**
Las medianas y los cuartiles se mantienen bastante estables a lo largo de las últimas 4 temporadas. Destacan algunos outliers como el de un jugador en la 18/19 que se verá más adelante.

#### Scatterplot

In [ ]:
fig = px.scatter(df_filt, x="ast", y="pts", color="season",
                 size='reb',hover_name="player_name",trendline='ols')

fig.add_annotation(x=10, y=34,
            text="Los jugadores más destacados",
                  showarrow = False)

fig.update_layout(
    title="Comparación estadística por jugadores en las últimas 4 temporadas",
    xaxis=dict(
        title='Asistencias',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Puntos',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0.02,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    )
)

fig.show()

**Conclusiones:**
Se puede observar que la mayor parte de los jugadores tienen valores bastante bajos en cuanto a medias de puntos y asistencias. Esto a su vez no implica que tengan malos números reboteadores, ya que hay varios puntos con números altos en rebotes y bajos en las otras dos métricas. Este tipo de jugadores suelen ser pivots altos que destacan por el trabajo defensivo y no tanto por el ofensivo, en el cual destacan las otras dos variables. Las líneas de regresión de las 4 temporadas indican claramente que hay una relación lineal positiva entre el número de asistencias y el número de puntos.

## Gráfico de barras del número de jugadores por equipo en los últimos 4 años

In [ ]:
df["NAC_USA"] = df["country"]

In [ ]:
localizacion = df[df["NAC_USA"]!="USA"].index
df.loc[localizacion,"NAC_USA"] = "Extranjeros"

In [ ]:
df_equipos = df_filt.groupby(["team_abbreviation","NAC_USA"])["player_name"].count().to_frame()
df_equipos.reset_index(inplace=True)  
df_equipos = df_equipos.pivot_table(values='player_name', index='team_abbreviation', columns='NAC_USA')
df_equipos = df_equipos.reset_index()
df_equipos["Total"] = df_equipos["Extranjeros"]+df_equipos["USA"]
df_equipos = df_equipos.sort_values("Total", ascending = False)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(x=df_equipos.team_abbreviation,
                y=df_equipos.Total,
                name='Total'
                ))
fig.add_trace(go.Bar(x=df_equipos.team_abbreviation,
                y=df_equipos.USA,
                name='Estadounidenses'
                ))
fig.add_trace(go.Bar(x=df_equipos.team_abbreviation,
                y=df_equipos.Extranjeros,
                name='Extranjeros'
                ))

fig.update_layout(
    title="Número de jugadores estadounidenses y extranjeros de cada equipo las 4 últimas temporadas",
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad de jugadores',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0.8,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group'
)
fig.show()

**Conclusiones:**
El número total de jugadores por equipo es bastante parejo, aunque destaca portland por tener plantillas cortas y Memphis al contrario. En cuanto al número de extranjeros en los equipos, Dallas, Utah, Phoenix y San Antonio han sido los equipos que mayor número han tenido los últimos 4 años.

# Comparar los jugadores más anotadores y asistentes de las 4 últimas temporadas

Se cogen de cada una de las últimas 4 temporadas los 4 jugadores más anotadores

In [ ]:
df_topjugadorespts = pd.DataFrame()

for i in seasons:
    df_filtseason = df_filt[df_filt["season"] == i]
    df_topjugadorespts = df_topjugadorespts.append(df_filtseason.sort_values("pts", ascending=False).iloc[:4]) #los 4 que más puntúan de cada temporada

In [ ]:
jugadores_top_pts = df_topjugadorespts.player_name.value_counts().iloc[:5].index.values #los que más aparecen
df_jugadores_pts = df_filt[df_filt["player_name"].isin(jugadores_top_pts)]

In [ ]:
df_jugadores_pts = df_jugadores_pts.groupby(["season", "player_name"])["pts"].mean().reset_index()
fig = px.density_heatmap(df_jugadores_pts, 
                        y='player_name',
                        x='season', 
                        z='pts', color_continuous_scale = 'Blues')
fig.update_xaxes(side="bottom")
fig.update_layout(title="Jugadores más anotadores a lo largo de las últimas 4 temporadas")
fig.show()

**Conclusiones:**
Los 5 jugadores tienen medias muy altas. Destaca James Harden, el jugador del outlier que se ha identificado al comienzo en los boxplots.

Se cogen de cada una de las últimas 4 temporadas los 4 jugadores más anotadores

In [ ]:
df_topjugadoresast = pd.DataFrame()

for i in seasons:
    df_filtseason = df_filt[df_filt["season"] == i]
    df_topjugadoresast = df_topjugadoresast.append(df_filtseason.sort_values("ast", ascending=False).iloc[:4])

In [ ]:
jugadores_top_ast = df_topjugadoresast.player_name.value_counts().iloc[:5].index.values #los que más aparecen
df_jugadores_ast = df_filt[df_filt["player_name"].isin(jugadores_top_ast)]

In [ ]:
df_jugadores_ast = df_jugadores_ast.groupby(["season", "player_name"])["ast"].mean().reset_index()
fig = px.density_heatmap(df_jugadores_ast, 
                        y='player_name',
                        x='season', 
                        z='ast', color_continuous_scale = 'Blues')
fig.update_xaxes(side="bottom")
fig.update_layout(title="Jugadores más asistentes a lo largo de las últimas 4 temporadas")
fig.show()

**Conclusiones:**
En este gráfico están algo más repartidos los valores al no existir un outlier muy claro.

# Mapa de los países de procedencia según la media de puntos

In [ ]:
import pycountry

input_countries = df[df["country"]!='USA'].country.unique()

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

codes = [countries.get(country, 'Unknown code') for country in input_countries]

In [ ]:
paises = pd.DataFrame(codes, df[df["country"]!='USA'].country.unique()).reset_index()
paises.columns = ["country", "country_map"]

In [ ]:
df_mapa = df.groupby("country")["pts"].mean().to_frame().reset_index()
df_mapa.columns = ["country", "media_pts"]
df_mapa = df_mapa.merge(paises, on = "country", how="left")

In [ ]:
loc = df_mapa[df_mapa["country"] == "USA"].index
df_mapa.loc[loc, "country_map"] = "USA"
df_mapa = df_mapa[df_mapa["country_map"]!= "Unknown code"]

In [ ]:
fig = px.choropleth(df_mapa, locations="country_map",
                    color="media_pts", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale='GnBu')

fig.update_layout(title="Mapa de los países de procedencia según la media de puntos")

fig.show()

**Conclusiones:**
Es muy curioso como los jugadores de países europeos tienen medias bastante altas en comparación con los jugadores estadounidenses. Esto significa que cuando se fichan jugadores europeos se trata de estrellas que luego tienen un gran impacto en la liga.

# Gráfico de araña para ver las estadísticas de los 3 mayores anotadores de los últimos 4 años

In [ ]:
df_topjugadorespts = pd.DataFrame()

for i in seasons:
    df_filtseason = df_filt[df_filt["season"] == i]
    df_topjugadorespts = df_topjugadorespts.append(df_filtseason.sort_values("pts", ascending=False).iloc[:4]) #los 4 que más puntúan de cada temporada

jugadores_top_pts = df_topjugadorespts.player_name.value_counts().iloc[:3].index.values #los que más aparecen
df_jugadores_pts = df_filt[df_filt["player_name"].isin(jugadores_top_pts)]

df_jugadores_pts = df_jugadores_pts.groupby("player_name")["pts", "reb","ast", "net_rating"].mean().reset_index()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=df_jugadores_pts[df_jugadores_pts["player_name"]=="Anthony Davis"].values[0][1:5],
      theta=df_jugadores_pts.columns[1:5],
      fill='toself',
      name='Davis'
))
fig.add_trace(go.Scatterpolar(
      r=df_jugadores_pts[df_jugadores_pts["player_name"]=="Giannis Antetokounmpo"].values[0][1:5],
      theta=df_jugadores_pts.columns[1:5],
      fill='toself',
      name='Giannis'
)),
fig.add_trace(go.Scatterpolar(
      r=df_jugadores_pts[df_jugadores_pts["player_name"]=="James Harden"].values[0][1:5],
      theta=df_jugadores_pts.columns[1:5],
      fill='toself',
      name='Harden'
))

fig.update_layout(
    title = "Comparativa estadística de los 3 mayores anotadores de la NBA",
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 35]
    )),
  showlegend=True
)

fig.show()

**Conclusiones:**
Los 3 jugadores tienen medias muy elevadas en cualquiera de las estadísticas, pero Harden se diferencia de los dos anteriores por su alta media de puntuación. Davis y Giannis son quizás más commpletos, ya que capturan muchos rebotes, pero Harden también destaca por encima de ambos en cuanto a asistencias, por lo que se trata de un perfil muy distinto. Cabe destacar que Giannis es el que mayor impacto positivo tiene en su equipo, dado que tiene el mayor valor en la variable net_rating, que describe la diferencia de puntos a favor o en contra en 100 posesiones con el jugador sobre el campo.